<a href="https://colab.research.google.com/github/anshudaur/TensorFlowProjects/blob/master/MNIST_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datasets import MNISTDataset
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

def main():
	# load data
	(X_train, y_train), (X_test, y_test) = mnist.load_data(plt.imshow(train_images[0], cmap="Greys_r")
	print("input dataset shape : " , X_train.shape,y_train.shape ,X_test.shape, y_test.shape)
	# flatten 28*28 images to a 784 vector for each image
	num_pixels = X_train.shape[1] * X_train.shape[2]
  X_train = X_train.reshape((X_train.shape[0], num_pixels)).astype('float32')
	X_test = X_test.reshape((X_test.shape[0], num_pixels)).astype('float32')
	# normalize inputs from 0-255 to 0-1
	X_train = X_train / 255
	X_test = X_test / 255
	# one hot encode outputs
	y_train = np_utils.to_categorical(y_train)
	y_test = np_utils.to_categorical(y_test)
	num_classes = y_test.shape[1]
	# define baseline model
	def baseline_model():
		# create model
		model = Sequential()
		model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
		model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
		# Compile model
		model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
		return model
	# build the model
	model = baseline_model()
	# Fit the model
	model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
	# Final evaluation of the model
	scores = model.evaluate(X_test, y_test, verbose=0)
	print("Baseline Error: %.2f%%" % (100-scores[1]*100))'''

if __name__ == '__main__':
    main();

IndentationError: ignored

# New Section